In [1]:
!pip install -U sentence-transformers tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.2
    Uninstalling tqdm-4.67.2:
      Successfully uninstalled tqdm-4.67.2


In [2]:
import json
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
!git clone https://github.com/Bhuvilol/CAIR-CD.git

Cloning into 'CAIR-CD'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 87 (delta 25), reused 74 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (87/87), 16.38 MiB | 11.30 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [4]:
import os

os.chdir('CAIR-CD')
print(f"Current working directory: {os.getcwd()}")

Current working directory: /content/CAIR-CD


In [5]:
jsonl_path = "EDA/clean_conversational_dataset.jsonl"
print(f"jsonl_path set to: {jsonl_path}")

jsonl_path set to: EDA/clean_conversational_dataset.jsonl


In [6]:
import json

conversations = []
conversation_ids = []

with open(jsonl_path, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        obj = json.loads(line)

        # Use metadata ID if available, else fallback to index
        cid = obj.get("metadata", {}).get("conversation_id", f"conv_{idx}")
        text = obj["input"]   # THIS is the conversation text

        conversation_ids.append(cid)
        conversations.append(text)

print(len(conversations), "conversations loaded")

1939 conversations loaded


In [7]:
embeddings = model.encode(
    conversations,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True
)

embeddings.shape

Batches:   0%|          | 0/122 [00:00<?, ?it/s]

(1939, 768)

In [8]:
import numpy as np, os, json

os.makedirs("artifacts", exist_ok=True)

np.save("artifacts/mpnet_embeddings.npy", embeddings)

with open("artifacts/embedding_index.json", "w") as f:
    json.dump(
        {cid: idx for idx, cid in enumerate(conversation_ids)},
        f,
        indent=2
    )

print("✅ mpnet embeddings saved successfully")

✅ mpnet embeddings saved successfully


In [9]:
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [10]:
import re
import unicodedata

def canonicalize(text: str) -> str:
    # unicode normalize
    text = unicodedata.normalize("NFKC", text)

    # lowercase
    text = text.lower()

    # normalize whitespace
    text = re.sub(r"\s+", " ", text)

    # strip
    return text.strip()

In [11]:
text_to_embedding = {}

with open("EDA/clean_conversational_dataset.jsonl", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        obj = json.loads(line)
        canon_text = canonicalize(obj["input"])
        text_to_embedding[canon_text] = embeddings[idx]

print("Indexed embeddings:", len(text_to_embedding))

Indexed embeddings: 1


In [12]:
def build_conversation_text(convo):
    raw = " ".join(
        f"{turn['speaker']}: {turn['text']}"
        for turn in convo
    )
    return canonicalize(raw)

In [13]:
import json

def build_conversation_text(convo):
    return " ".join(
        f"{turn['speaker']}: {turn['text']}"
        for turn in convo
    )

all_texts = []
all_labels = []

with open("EDA/clean_conversational_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

for obj in data["transcripts"]:
    text = build_conversation_text(obj["conversation"])
    all_texts.append(text)
    all_labels.append(obj["intent"])

len(all_texts)

1939

In [14]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

embeddings = model.encode(
    all_texts,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True
)

embeddings.shape

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/122 [00:00<?, ?it/s]

(1939, 768)

In [15]:
import os
os.makedirs("artifacts", exist_ok=True)

np.save("artifacts/mpnet_embeddings.npy", embeddings)


In [16]:
id_to_index = {}

for idx, obj in enumerate(data["transcripts"]):
    id_to_index[obj["transcript_id"]] = idx

import json
with open("artifacts/id_to_index.json", "w") as f:
    json.dump(id_to_index, f, indent=2)


In [17]:
import numpy as np

def build_split(split_path):
    X, y = [], []

    with open(split_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    for obj in data["transcripts"]:
        idx = id_to_index[obj["transcript_id"]]
        emb = embeddings[idx]

        total_turns = len(obj["conversation"])
        customer_turns = sum(
            1 for t in obj["conversation"] if t["speaker"] == "Customer"
        )

        feats = [
            total_turns,
            customer_turns / total_turns if total_turns else 0.0
        ]

        X.append(np.concatenate([feats, emb]))
        y.append(obj["intent"])

    return np.array(X), np.array(y)


In [18]:
X_train, y_train = build_split("EDA/clean_train_dataset.json")
X_val, y_val     = build_split("EDA/clean_validation_dataset.json")
X_test, y_test   = build_split("EDA/clean_test_dataset.json")

print(X_train.shape, X_val.shape, X_test.shape)


(1357, 770) (291, 770) (291, 770)


In [19]:
assert X_train.shape[0] > 0
assert X_train.shape[1] == X_val.shape[1] == X_test.shape[1]


In [20]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
import numpy as np


In [21]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc   = le.transform(y_val)
y_test_enc  = le.transform(y_test)

num_classes = len(le.classes_)
print("Classes:", le.classes_)


Classes: ['Account Access Issues' 'Appointment Scheduling' 'Claim Denials'
 'Delivery Investigation' 'Escalation - Repeated Service Failures'
 'Escalation - Threat of Legal Action' 'Fraud Alert Investigation'
 'Multiple Issues - Order Status, Billing & Account'
 'Multiple Issues - Returns & Account Inquiries' 'Service Interruptions']


In [22]:
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │       197,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 231,562 (904.54 KB)

 Trainable params: 231,562 (904.54 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model.fit(
    X_train, y_train_enc,
    validation_data=(X_val, y_val_enc),
    epochs=20,
    batch_size=32,
    verbose=1
)


Epoch 1/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2114 - loss: 2.2220 - val_accuracy: 0.8660 - val_loss: 1.4196
Epoch 2/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7546 - loss: 1.2363 - val_accuracy: 0.9966 - val_loss: 0.3193
Epoch 3/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9668 - loss: 0.3656 - val_accuracy: 0.9966 - val_loss: 0.0672
Epoch 4/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9869 - loss: 0.1156 - val_accuracy: 0.9966 - val_loss: 0.0333
Epoch 5/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9890 - loss: 0.0747 - val_accuracy: 0.9966 - val_loss: 0.0178
Epoch 6/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9936 - loss: 0.0431 - val_accuracy: 0.9966 - val_loss: 0.0110
Epoch 7/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9976 - loss: 0.0284 - val_accuracy: 0.9966 - val_loss: 0.0087
Epoch 8/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9994 - loss: 0.0175 - val_accuracy: 1.0000 - val_l

In [24]:
test_loss, test_acc = model.evaluate(X_test, y_test_enc, verbose=0)
print("Test Accuracy:", round(test_acc, 4))


Test Accuracy: 1.0


In [25]:
import os, json

os.makedirs("artifacts", exist_ok=True)

model.save("artifacts/outcome_predictor.keras")

with open("artifacts/label_mapping.json", "w") as f:
    json.dump(
        {i: label for i, label in enumerate(le.classes_)},
        f,
        indent=2
    )

print("✅ Model and label mapping saved")


✅ Model and label mapping saved


In [26]:
from tensorflow import keras

# Save model to Colab filesystem
model.save("/content/outcome_predictor.keras")

print("✅ Model saved at /content/outcome_predictor.keras")


✅ Model saved at /content/outcome_predictor.keras


In [27]:
from google.colab import files

files.download("/content/outcome_predictor.keras")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>